# Dependency code

In [ ]:
import imp
import os
import sys
import numpy as np
import glob
import cmdbench

fp, pathname, description = imp.find_module('benchmark', ['../lib'])
benchmark = imp.load_module('benchmark', fp, pathname, description)

# Summarize numpy array if it has more than 10 elements
np.set_printoptions(threshold=10)

# Software versions

In [ ]:
!mash --version

# Benchmarking

## Input data and constants

In [ ]:
input_dir = '../data/input-files/reads'
input_files_1 = [os.path.basename(f) for f in glob.glob(f'{input_dir}/*_1.fastq.gz')]
input_samples = [f.replace('_1.fastq.gz','') for f in input_files_1]
input_samples.sort()
print(input_samples)

nproc=32
kmer=31
sketch_size=100000

mash_dir = 'mash-data'

sample_sizes = [1,10,20,30,40,50,60,70,80]

benchmark.create_folder_if_doesnt_exist(mash_dir)

## Benchmark functions

In [ ]:
def reset_func():
    benchmark.clean_if_exists(mash_dir)
    
def sampling_func(sample_size):
    samples = input_samples[:sample_size]
    return samples
    
mash_command = {
    "command": f"mash sketch -r -p 1 -s {sketch_size} -k {kmer} -I % -o {mash_dir}/%.msh {input_dir}/%_1.fastq.gz {input_dir}/%_2.fastq.gz",
    "parallel_args": f"-j {nproc} -I%",
}

combine_sketchs_command = {
    "command": lambda inputs: f"mash paste {mash_dir}/sketches " + ' '.join([f"{mash_dir}/{x}.msh" for x in inputs]),
}

dist_calculation_times = 1
dist_calculating_genome = "2014C-3600_1.fastq.gz"

query_command = {
    "command": f"mash dist -p {nproc} {mash_dir}/sketches.msh {input_dir}/%",
    "parallel_args": "-j 1 -I%",
    "parallel_argfiles": [dist_calculating_genome] * dist_calculation_times
}

## Benchmark

In [ ]:
multibench_results, debug_str = benchmark.multibench.multi_cmdbench({
        "index": [mash_command, combine_sketchs_command],
        "query": [query_command]
    },
    reset_func = reset_func, iterations = 1, sampling_func = sampling_func, sample_sizes = sample_sizes, 
    benchmark_list_to_results=benchmark.benchmark_list_to_results, active_output_print = True
)

# Save and reload results

In [ ]:
save_path = "mash-results.txt"

samples_per_sample_size = []
for sample_size in sample_sizes:
        samples_per_sample_size.append(input_samples[:sample_size])

benchmark.multibench.save_multibench_results(multibench_results, samples_per_sample_size, save_path)
multibench_results, samples_per_sample_size = benchmark.multibench.read_multibench_results(save_path)

# Plot

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np
from pylab import rcParams
rcParams['figure.figsize'] = 15, 3

In [ ]:
# Indexing Plots
benchmark.multibench.plot_resources(multibench_results, sample_sizes, "index")

In [ ]:
# Querying Plots
benchmark.multibench.plot_resources(multibench_results, sample_sizes, "query")